In [1]:
!pip install transformers datasets evaluate scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
t

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


**We will copy from our repository the functions we need!**

In [1]:
import glob
import html
import os

#from config import DATA_PATH #load path for data files in the file config
DATA_PATH = "/content/drive/MyDrive/NLP_PROJECT2"




SEPARATOR = "\t"


def clean_text(text):
    """
    Remove extra quotes from text files and html entities
    Argumentss:
        text (str): a string of text

    Returns: (str): the "cleaned" text

    """
    text = text.rstrip() #removes spaces, tabs, new lines etc FROM THE END of the text

    if '""' in text: #if we have useless double "  we clean them (""somthing"")
        if text[0] == text[-1] == '"':
            text = text[1:-1] #take the string without the first and last char (remove ")
        text = text.replace('\\""', '"')
        text = text.replace('""', '"') #double ""  replaced by "

    text = text.replace('\\""', '"')

    text = html.unescape(text) #convert HTML entities into characters ex. &lt; → <
    text = ' '.join(text.split()) #splits the text ignoring many spaces and ' '.join()
                                  # joins them with only one space
    return text


def parse_file(file):
    """
    Read a file and return a dictionary of the data, in the format:
    tweet_id:{sentiment, text}
    """

    data = {}
    lines = open(file, "r", encoding="utf-8").readlines() #read all lines of the file (utf-8 for special chars)
    for _, line in enumerate(lines):
        columns = line.rstrip().split(SEPARATOR) #seperator = \t =  tab so we get each column value which is seperated by tab
        tweet_id = columns[0]
        sentiment = columns[1]
        text = columns[2:] #everything from the 3rd column and after (text)
        text = clean_text(" ".join(text)) #we set text as an element not a list of elements and clean it
                                          #in other words text becomes a string instead of a list
        data[tweet_id] = (sentiment, text) #(emotion, text)
    return data


def load_from_dir(path):
    """
    Searches for all the .tsv and .txt files in a folder

    """

    #search inside folders and subfolders
    files = glob.glob(path + "/**/*.tsv", recursive=True)
    files.extend(glob.glob(path + "/**/*.txt", recursive=True))

    data = {}  # use dict, in order to avoid having duplicate tweets (same id)
               #dictionary will replace the duplicate key
    for file in files:
        file_data = parse_file(file)
        data.update(file_data)
    return list(data.values()) #list of tuples (sentiment, text) --> dont care about the keys





def load_Semeval2017A():
    """
    Loads data from dataset Semeval2017A
    """


    train = load_from_dir(os.path.join(DATA_PATH, "Semeval2017A/train_dev"))
    test = load_from_dir(os.path.join(DATA_PATH, "Semeval2017A/gold"))

    X_train = [x[1] for x in train]
    y_train = [x[0] for x in train]
    X_test = [x[1] for x in test]
    y_test = [x[0] for x in test]

    return X_train, y_train, X_test, y_test





def load_MR():
    with open(os.path.join(DATA_PATH, "MR/rt-polarity.pos"), encoding="utf-8", errors="replace") as f:
        pos = f.readlines()

    with open(os.path.join(DATA_PATH, "MR/rt-polarity.neg"), encoding="utf-8", errors="replace") as f:
        neg = f.readlines()

    pos = [x.strip() for x in pos]
    neg = [x.strip() for x in neg]

    pos_labels = ["positive"] * len(pos)
    neg_labels = ["negative"] * len(neg)

    split = 5000

    X_train = pos[:split] + neg[:split]
    y_train = pos_labels[:split] + neg_labels[:split]

    X_test = pos[split:] + neg[split:]
    y_test = pos_labels[split:] + neg_labels[split:]

    return X_train, y_train, X_test, y_test

# MAIN CODE


In [3]:
import numpy as np
import evaluate
import torch
from sklearn.metrics import accuracy_score
from datasets import Dataset
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import LabelEncoder

In [12]:
# ==== METRICS ====
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


# ==== TOKENIZATION ====
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)

def prepare_dataset(X, y):
    return Dataset.from_dict({'text': X, 'label': y})

# **We will start with MR dataset.**



In [5]:
# ==== LOAD DATA ====
DATASET = "MR"
X_train, y_train, X_test, y_test = load_MR()


# Label encoding
le = LabelEncoder()
le.fit(list(set(y_train)))
y_train = le.transform(y_train)
y_test = le.transform(y_test)
n_classes = len(le.classes_)

# Prepare datasets
train_set = prepare_dataset(X_train, y_train)
test_set = prepare_dataset(X_test, y_test)




**First model: siebert/sentiment-roberta-large-english**




In [6]:
import torch
torch.cuda.empty_cache()



PRETRAINED_MODEL = 'siebert/sentiment-roberta-large-english'
os.environ["WANDB_DISABLED"] = "true" #we asked for API key to have wandb -> visualization of training loss
                                      #so we disable it


# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels=n_classes)

# Tokenize data
tokenized_train_set = train_set.map(tokenize_function, batched=True)
tokenized_test_set = test_set.map(tokenize_function, batched=True)

# (Optional) Subsample για debug
n_samples = 100
small_train_dataset = tokenized_train_set.shuffle(seed=42).select(range(n_samples))
small_eval_dataset = tokenized_test_set.shuffle(seed=42).select(range(n_samples))


# ==== TRAINING SETUP ====
args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    logging_steps=10,
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# ==== FINAL EVALUATION ON FULL TEST SET ====
predictions = trainer.predict(tokenized_test_set)
pred_labels = np.argmax(predictions.predictions, axis=-1)
acc = accuracy_score(y_test, pred_labels)
print(f"Model: {PRETRAINED_MODEL}")
print(f"\nFinal Test Accuracy: {acc:.4f}")


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/662 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.689800,0.676394,0.700000
2,0.686600,0.647820,0.810000
3,0.345500,0.666962,0.820000
4,0.293700,0.324012,0.910000
5,0.248100,0.403266,0.890000


Model: siebert/sentiment-roberta-large-english

Final Test Accuracy: 0.8625


**Second model: distilbert-base-uncased-finetuned-sst-2-english**

In [7]:
import torch
torch.cuda.empty_cache()



PRETRAINED_MODEL = 'distilbert-base-uncased-finetuned-sst-2-english'
os.environ["WANDB_DISABLED"] = "true" #we asked for API key to have wandb -> visualization of training loss
                                      #so we disable it


# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels=n_classes)

# Tokenize data
tokenized_train_set = train_set.map(tokenize_function, batched=True)
tokenized_test_set = test_set.map(tokenize_function, batched=True)

# (Optional) Subsample για debug
n_samples = 100
small_train_dataset = tokenized_train_set.shuffle(seed=42).select(range(n_samples))
small_eval_dataset = tokenized_test_set.shuffle(seed=42).select(range(n_samples))


# ==== TRAINING SETUP ====
args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    logging_steps=10,
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# ==== FINAL EVALUATION ON FULL TEST SET ====
predictions = trainer.predict(tokenized_test_set)
pred_labels = np.argmax(predictions.predictions, axis=-1)
acc = accuracy_score(y_test, pred_labels)
print(f"Model: {PRETRAINED_MODEL}")
print(f"\nFinal Test Accuracy: {acc:.4f}")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/662 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.532600,0.397851,0.870000
2,0.281900,0.451669,0.870000
3,0.096900,0.547714,0.870000
4,0.001200,0.595341,0.870000
5,0.000800,0.606520,0.870000


Model: distilbert-base-uncased-finetuned-sst-2-english

Final Test Accuracy: 0.8958


Third model: textattack/bert-base-uncased-SST-2

In [8]:
import torch
torch.cuda.empty_cache()



PRETRAINED_MODEL = 'textattack/bert-base-uncased-SST-2'
os.environ["WANDB_DISABLED"] = "true" #we asked for API key to have wandb -> visualization of training loss
                                      #so we disable it


# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels=n_classes)

# Tokenize data
tokenized_train_set = train_set.map(tokenize_function, batched=True)
tokenized_test_set = test_set.map(tokenize_function, batched=True)

# (Optional) Subsample για debug
n_samples = 100
small_train_dataset = tokenized_train_set.shuffle(seed=42).select(range(n_samples))
small_eval_dataset = tokenized_test_set.shuffle(seed=42).select(range(n_samples))


# ==== TRAINING SETUP ====
args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    logging_steps=10,
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# ==== FINAL EVALUATION ON FULL TEST SET ====
predictions = trainer.predict(tokenized_test_set)
pred_labels = np.argmax(predictions.predictions, axis=-1)
acc = accuracy_score(y_test, pred_labels)
print(f"Model: {PRETRAINED_MODEL}")
print(f"\nFinal Test Accuracy: {acc:.4f}")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Map:   0%|          | 0/662 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.484900,0.482360,0.840000
2,0.277700,0.408721,0.870000
3,0.058900,0.478676,0.910000
4,0.000800,0.576896,0.880000
5,0.000500,0.592327,0.870000


Model: textattack/bert-base-uncased-SST-2

Final Test Accuracy: 0.8837


# **We will continue with Semeval dataset.**

In [9]:
DATASET = "Semeval2017A"
X_train, y_train, X_test, y_test = load_Semeval2017A()


# Label encoding
le = LabelEncoder()
le.fit(list(set(y_train)))
y_train = le.transform(y_train)
y_test = le.transform(y_test)
n_classes = len(le.classes_)

# Prepare datasets
train_set = prepare_dataset(X_train, y_train)
test_set = prepare_dataset(X_test, y_test)

**First model: cardiffnlp/twitter-roberta-base-sentiment**



In [13]:
import torch
torch.cuda.empty_cache()



PRETRAINED_MODEL = 'cardiffnlp/twitter-roberta-base-sentiment'
os.environ["WANDB_DISABLED"] = "true" #we asked for API key to have wandb -> visualization of training loss
                                      #so we disable it


# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels=n_classes)

# Tokenize data
tokenized_train_set = train_set.map(tokenize_function, batched=True)
tokenized_test_set = test_set.map(tokenize_function, batched=True)

# (Optional) Subsample για debug
n_samples = 100
small_train_dataset = tokenized_train_set.shuffle(seed=42).select(range(n_samples))
small_eval_dataset = tokenized_test_set.shuffle(seed=42).select(range(n_samples))


# ==== TRAINING SETUP ====
args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    logging_steps=10,
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# ==== FINAL EVALUATION ON FULL TEST SET ====
predictions = trainer.predict(tokenized_test_set)
pred_labels = np.argmax(predictions.predictions, axis=-1)
acc = accuracy_score(y_test, pred_labels)
print(f"Model: {PRETRAINED_MODEL}")
print(f"\nFinal Test Accuracy: {acc:.4f}")

Map:   0%|          | 0/49570 [00:00<?, ? examples/s]

Map:   0%|          | 0/12284 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.646000,0.721568,0.670000
2,0.529100,1.054714,0.620000
3,0.142700,1.753020,0.580000
4,0.010200,2.084121,0.580000
5,0.034100,2.048360,0.580000


Model: cardiffnlp/twitter-roberta-base-sentiment

Final Test Accuracy: 0.6903


**Second model: finiteautomata/bertweet-base-sentiment-analysis**

In [14]:
import torch
torch.cuda.empty_cache()



PRETRAINED_MODEL = 'finiteautomata/bertweet-base-sentiment-analysis'
os.environ["WANDB_DISABLED"] = "true" #we asked for API key to have wandb -> visualization of training loss
                                      #so we disable it


# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels=n_classes)

# Tokenize data
tokenized_train_set = train_set.map(tokenize_function, batched=True)
tokenized_test_set = test_set.map(tokenize_function, batched=True)

# (Optional) Subsample για debug
n_samples = 100
small_train_dataset = tokenized_train_set.shuffle(seed=42).select(range(n_samples))
small_eval_dataset = tokenized_test_set.shuffle(seed=42).select(range(n_samples))


# ==== TRAINING SETUP ====
args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    logging_steps=10,
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# ==== FINAL EVALUATION ON FULL TEST SET ====
predictions = trainer.predict(tokenized_test_set)
pred_labels = np.argmax(predictions.predictions, axis=-1)
acc = accuracy_score(y_test, pred_labels)
print(f"Model: {PRETRAINED_MODEL}")
print(f"\nFinal Test Accuracy: {acc:.4f}")

tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

Map:   0%|          | 0/12284 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.508500,0.943131,0.650000
2,0.308700,0.682151,0.760000
3,0.262800,1.058799,0.720000
4,0.025700,1.020295,0.740000
5,0.010000,1.042668,0.740000


Model: finiteautomata/bertweet-base-sentiment-analysis

Final Test Accuracy: 0.7077


**Third model: yiyanghkust/finbert-tone**

In [15]:
import torch
torch.cuda.empty_cache()



PRETRAINED_MODEL = 'yiyanghkust/finbert-tone'
os.environ["WANDB_DISABLED"] = "true" #we asked for API key to have wandb -> visualization of training loss
                                      #so we disable it


# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels=n_classes)

# Tokenize data
tokenized_train_set = train_set.map(tokenize_function, batched=True)
tokenized_test_set = test_set.map(tokenize_function, batched=True)

# (Optional) Subsample για debug
n_samples = 100
small_train_dataset = tokenized_train_set.shuffle(seed=42).select(range(n_samples))
small_eval_dataset = tokenized_test_set.shuffle(seed=42).select(range(n_samples))


# ==== TRAINING SETUP ====
args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=8,
    logging_steps=10,
    save_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

# ==== FINAL EVALUATION ON FULL TEST SET ====
predictions = trainer.predict(tokenized_test_set)
pred_labels = np.argmax(predictions.predictions, axis=-1)
acc = accuracy_score(y_test, pred_labels)
print(f"Model: {PRETRAINED_MODEL}")
print(f"\nFinal Test Accuracy: {acc:.4f}")

config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Map:   0%|          | 0/49570 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Map:   0%|          | 0/12284 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,2.824800,1.337254,0.430000
2,1.073500,1.671640,0.280000
3,0.686800,1.825149,0.440000
4,0.133500,2.011916,0.500000
5,0.087500,2.121940,0.420000


Model: yiyanghkust/finbert-tone

Final Test Accuracy: 0.4141
